In [15]:
"""
1️⃣ partial — freeze arguments like a pro
Exercise 1.1

You’re given a generic logger:

def log(level, module, message):
    return f"[{level}] {module}: {message}"


Tasks

Create error_logger that always logs at "ERROR" level.

Create auth_error_logger that logs "ERROR" level from "AUTH" module.

Use it like:

auth_error_logger("Invalid token")


Interview signal

You understand function currying

You avoid lambdas for readability & reuse
"""

from functools import partial

def log(level, module, message):
    print( f"[{level}] {module}: {message}" )

error_logger = partial(log, "ERROR")
auth_error_logger = partial(log, "ERROR", "AUTH" )

log("INFO","MAIN", "Program Started")
error_logger("MAIN", "Unable to find variable c")
auth_error_logger("Invalid Token")

[INFO] MAIN: Program Started
[ERROR] MAIN: Unable to find variable c
[ERROR] AUTH: Invalid Token


In [13]:
"""
2️⃣ partialmethod — same idea, but for classes
Exercise 2.1

You’re building a notification system:

class Notifier:
    def send(self, channel, message):
        return f"Sending '{message}' via {channel}"


Tasks

Add methods:

send_email

send_sms

DO NOT duplicate logic.

Use partialmethod.

Expected usage

n = Notifier()
n.send_email("Hello")


Interview signal

You understand descriptor behavior

You know why partial is not enough in classes
"""
from functools import partialmethod

class Notifier:
    def send(self, channel, message):
        print(f"Sending '{message}' via {channel}")

    send_email = partialmethod(send, "email")
    send_sms = partialmethod(send, "sms")

n = Notifier()
n.send_email(message="Hello")
n.send_sms(message="Hi")

Sending 'Hello' via email
Sending 'Hi' via sms


In [21]:
"""
3️⃣ reduce — when loops are not expressive enough
Exercise 3.1

Given a list of dictionaries:

orders = [
    {"item": "apple", "qty": 2},
    {"item": "apple", "qty": 3},
    {"item": "banana", "qty": 5},
]


Task
Use reduce to produce:

{"apple": 5, "banana": 5}


🚫 No defaultdict
🚫 No for loop

Interview signal

You understand accumulator patterns

You can reason about immutability vs mutation
"""

from functools import reduce

orders = [
    {"item": "apple", "qty": 2},
    {"item": "apple", "qty": 3},
    {"item": "banana", "qty": 5},
]

def accumulate(res, order):
    item = order['item']
    qty = order['qty']

    res[item] = res.get('qty',0) + qty
    return res

reduce(accumulate, orders, {})

{'apple': 3, 'banana': 5}

In [28]:
"""
Built-in all(iterable):

Returns True if every element is truthy

Stops early if it finds a falsy value

Empty iterable → True (important edge case!)

Examples
all([True, True, True])        # True
all([True, False, True])      # False
all([1, 2, 3])                # True
all([1, 0, 3])                # False
all([])                       # True

🎯 Goal of Exercise 3.2

Reimplement all() using functools.reduce.

Constraints:

❌ No loops

❌ No calling all()

✅ Use reduce
"""
z=[True, False, True]
reduce(lambda x, y: bool(x) and bool(y), z, True)

False

In [35]:
"""
4️⃣ cmp_to_key — custom sorting logic
Exercise 4.1

You’re sorting version numbers:

versions = ["1.10.2", "1.2.9", "1.2.10", "2.0.1"]


Task
Sort them correctly:

["1.2.9", "1.2.10", "1.10.2", "2.0.1"]


Rules

You MUST use cmp_to_key

You may not pre-convert versions

Interview signal

You understand Python’s sorting internals

You know when key= is not enough
"""
from functools import cmp_to_key
def compare_versions(v1, v2):
    major1, minor1, patch1 = map(int, v1.split("."))
    major2, minor2, patch2 = map(int, v2.split("."))
    if major1 > major2:
        return 1
    elif major1 < major2:
        return -1
    else:
        # if equal go the next part
        if minor1 > minor2:
            return 1
        elif minor1 < minor2:
            return -1
        else:
            # if minor is also equal go to patch
            if patch1 > patch2:
                return 1
            elif patch1 < patch2:
                return -1
            else:
                return 0

versions = ["1.10.2", "1.2.9", "1.2.10", "2.0.1"]
res = sorted(versions, key=cmp_to_key(compare_versions))
print(res)

['1.2.9', '1.2.10', '1.10.2', '2.0.1']


In [40]:
"""
5️⃣ total_ordering + __lt__
Exercise 5.1

Create a Job class:

Job(priority, timestamp)


Sorting rules

Higher priority first

If priority same → earlier timestamp first

Tasks

Implement only __eq__ and __lt__

Use @total_ordering

Show that >, <=, >= all work

Interview signal

You understand comparison contracts

You write minimal, correct magic methods
"""
import time
from datetime import datetime
from functools import total_ordering

@total_ordering
class Job:

    def __init__(self, priority, timestamp):
        self.priority = priority
        self.timestamp = timestamp

    def __eq__(self, jobj):
        return self.priority == jobj.priority and self.timestamp == jobj.timestamp

    def __lt__(self, jobj):
        if self.priority == jobj.priority:
            return self.timestamp < jobj.timestamp
        return self.priority > jobj.priority

j1 = Job(1, datetime.now())
j2 = Job(5, datetime.now())

print(j1 < j2)
print(j1 <= j2)
print(j1 > j2)
print(j1 >= j2)
print(j1 == j2)

False
False
True
True
False


In [44]:
"""
6️⃣ update_wrapper — manual decorator hygiene
Exercise 6.1

Write a decorator:

def debug(fn):
    def wrapper(*args, **kwargs):
        print("Calling", fn.__name__)
        return fn(*args, **kwargs)
    return wrapper


Task
Fix metadata issues:

debugged_fn.__name__
debugged_fn.__doc__


🚫 Do NOT use @wraps yet
✅ Use update_wrapper

Interview signal

You know what decorators break

You can fix it without magic
"""

from functools import update_wrapper

def decorater(func):
    def decoratee(*args, **kwargs):
        print(f"Calling function {func.__name__}")
        return func(*args, **kwargs)
    print(f"Called function {decoratee.__name__}")
    update_wrapper(decoratee, func)
    print(f"Called function {decoratee.__name__}")
    return decoratee

@decorater
def myfunc(name, msg):
    print(f"Hello {name}, {msg}")

myfunc("Hema", "Welcome")

Called function decoratee
Called function myfunc
Calling function myfunc
Hello Hema, Welcome


In [49]:
"""
7️⃣ wraps — the right way™
Exercise 7.1

Rewrite Exercise 6.1 using @wraps.

Bonus

Stack two decorators and prove metadata still works.
"""

from functools import wraps

def decorator1(func):
    @wraps(func)
    def decoratee(*args, **kwargs):
        print(f"Adding coffee for {decoratee.__name__}")
        return func(*args, **kwargs)
    return decoratee

def decorator2(func):
    @wraps(func)
    def decoratee(*args, **kwargs):
        print(f"Adding mocha for {decoratee.__name__}")
        return func(*args, **kwargs)
    return decoratee
    
@decorator1
@decorator2
def milk():
    print("Having a nice cup of milk!")

milk()

Adding coffee for milk
Adding mocha for milk
Having a nice cup of milk!


In [59]:
"""
🧩 Simple lru_cache Coding Exercise
Problem: Expensive greeting formatter

You’re given a function that formats a greeting message.

Assume formatting is expensive (think: template rendering, validation, etc.).

Step 1️⃣ Starter code (no cache)
def format_greeting(name):
    print("Formatting greeting for:", name)
    return f"Hello, {name.strip().title()}!"


Run this:

format_greeting("hema")
format_greeting("hema")
format_greeting("hema")

Expected output
Formatting greeting for: hema
Formatting greeting for: hema
Formatting greeting for: hema

Step 2️⃣ Your task

Modify the function so that:

Repeated calls with the same name

Do not re-run the function body

Use @lru_cache

Step 3️⃣ Test case (must match)
format_greeting("hema")
format_greeting("hema")
format_greeting("alex")
format_greeting("hema")

Expected output
Formatting greeting for: hema
Formatting greeting for: alex


(Only two real executions)
"""

import time

def format_greeting(name):
    return f"Hello, {name.strip().title()}!"

start = time.time()
res = [format_greeting("hema") for _ in range(1000)]
print(f"Time taken: {time.time()-start} seconds")

from functools import lru_cache

@lru_cache(maxsize=None)
def format_greeting1(name):
    return f"Hello, {name.strip().title()}!"

start = time.time()
res = [format_greeting1("hema") for _ in range(1000)]
print(f"Time taken: {time.time()-start} seconds")
print(f"Cache info: {format_greeting1.cache_info()}")

Time taken: 0.0007150173187255859 seconds
Time taken: 0.0004482269287109375 seconds
Cache info: CacheInfo(hits=999, misses=1, maxsize=None, currsize=1)


In [72]:
"""
9️⃣ singledispatch — clean polymorphism
Exercise 9.1

Create a function:

def serialize(obj):
    ...


Support

dict → JSON-like string

list → comma-separated

int → hex string

default → str(obj)

Use @singledispatch.

Interview signal

You understand open/closed principle

You know how Python does function overloading
"""
import json
from functools import singledispatch

@singledispatch
def serialize(obj: str):
    return obj

@serialize.register(list)
def _(obj: list):
    return ",".join(obj)

@serialize.register(dict)
def _(obj: dict):
    return json.dumps(obj)

@serialize.register(int)
def _(obj: int):
    return hex(obj)

@serialize.register(set)
def _(obj: set):
    return ",".join(obj)

print(serialize("Hello"))
print(serialize(list(map(str, [1,2,3,4]))))
print(serialize({"a":1,"b":2}))
print(serialize(1234))
print(serialize((set(map(str,{1,1,3,4,4,5,6,6})))))



Hello
1,2,3,4
{"a": 1, "b": 2}
0x4d2
1,4,5,3,6
